# Chap3 Deep Learning with Pytorch

## Tensors

### The creation of tensors

In [4]:
import torch
import numpy as np

In [5]:
a = torch.FloatTensor(3, 2)
a

tensor([[6.3058e-44, 6.7262e-44],
        [7.1466e-44, 6.3058e-44],
        [6.7262e-44, 7.2868e-44]])

In [6]:
a.zero_()

tensor([[0., 0.],
        [0., 0.],
        [0., 0.]])

### GPU tensors

In [7]:
a = torch.FloatTensor([2, 3])
a

tensor([2., 3.])

In [8]:
ca = a.to('cuda')
ca

tensor([2., 3.], device='cuda:0')

In [9]:
a + 1

tensor([3., 4.])

In [10]:
ca + 1

tensor([3., 4.], device='cuda:0')

In [11]:
ca.device

device(type='cuda', index=0)